# Twitch PostgreSQL database

### PostgreSQL connection

Based on https://naysan.ca/2020/05/31/postgresql-to-pandas/

In [1]:
# Librairies
import psycopg2
import pandas as pd
import numpy as np
import sys

In [2]:
# Connection parameters to login
co_param = {
    "host"      : "twitch.caampywfg0rz.us-east-1.rds.amazonaws.com",
    "database"  : "Twitch",
    "user"      : "GaTech_team_96",
    "password"  : "i-love-my-coffee-without-milk-and-sugar-at-800AM"
}

In [3]:
def connect(co_param):
    """
    Connect to the PostgreSQL database server
    """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**co_param)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn

In [4]:
def postgresql_to_dataframe(conn, select_query, column_names):
    """
    Tranform a SELECT query into a pandas dataframe
    """
    cursor = conn.cursor()
    try:
        cursor.execute(select_query)
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        cursor.close()
        return 1
    
    # Naturally we get a list of tupples
    tupples = cursor.fetchall()
    cursor.close()
    
    # We just need to turn it into a pandas dataframe
    df = pd.DataFrame(tupples, columns=column_names)
    return df

### stream_data database

In [5]:
# SQL query
sql_query = """SELECT * FROM stream_data"""

# Column names
stream_data_col_names = ["game_id","stream_id","language","started_at","title",
                            "stream_type","user_id","user_name","viewer_count","user_login","game_name",
                            "thumbnail_url","tag_ids","is_mature","time_logged"]

# Retrieving the data
stream_data = postgresql_to_dataframe(connect(co_param), sql_query, stream_data_col_names)
stream_data.head()

Connecting to the PostgreSQL database...
Connection successful


,game_id,stream_id,language,started_at,title,stream_type,user_id,user_name,viewer_count,user_login,game_name,thumbnail_url,tag_ids,is_mature,time_logged
0,497057,40052420184,en,2022-11-09T11:02:36Z,Didn't Get Ghost Writer KEKW | OW2 Later!,live,96828357,Gumpita,60,gumpita,Destiny 2,https://static-cdn.jtvnw.net/previews-ttv/live...,{6ea6bca4-4712-4ab9-a906-e3336a9d8039},False,2022-11-09 12:25:26
1,497057,40052300424,en,2022-11-09T10:03:38Z,"6 GMs, 3 Guardians, 1 stream. LET'S GO. // !ads",live,54943031,RoboticAdi,44,roboticadi,Destiny 2,https://static-cdn.jtvnw.net/previews-ttv/live...,{6ea6bca4-4712-4ab9-a906-e3336a9d8039},True,2022-11-09 12:25:26
2,497057,46278752252,en,2022-11-09T11:24:23Z,GMs GMs GMs Help !wraith !merch !secretlab,live,126780483,GrandadGaming_,43,grandadgaming_,Destiny 2,https://static-cdn.jtvnw.net/previews-ttv/live...,{6ea6bca4-4712-4ab9-a906-e3336a9d8039},True,2022-11-09 12:25:26
3,497057,41479595547,en,2022-11-09T07:49:46Z,PvE n chill !throne !adorabullets,live,584365733,TalkQwertyToMe_,43,talkqwertytome_,Destiny 2,https://static-cdn.jtvnw.net/previews-ttv/live...,{6ea6bca4-4712-4ab9-a906-e3336a9d8039},True,2022-11-09 12:25:26
4,497057,40052489192,de,2022-11-09T11:33:33Z,Solo Flawless Großmeister Sog der Habsucht | !...,live,98447720,iPhil_,38,iphil_,Destiny 2,https://static-cdn.jtvnw.net/previews-ttv/live...,{9166ad14-41f1-4b04-a3b8-c8eb838c6be6},True,2022-11-09 12:25:26


###

0. filtering at english game
1. temp => get the diff of time_logged and started_at
2. how is the game behaving after 1week or 2? >>> defining a target for the MLalgo
        input target
        avg/medium/total viewers >>> 
3. Post-temp >>> sentiment analysis >>> title
4. Audience target > is_mature of the game
5. python dict for the categories of the game



In [6]:
# Generating a temp database
df_temp = stream_data.copy()

# changing formatting from 
df_temp.loc[:,"started_at"] = df_temp.loc[:,"started_at"].map(lambda x: x.rstrip("Z"))
df_temp.loc[:,"started_at"] = df_temp.loc[:,"started_at"].map(lambda x: x.replace("T", " "))
df_temp["stream_duration_hours"] = pd.to_datetime(df_temp["time_logged"])-pd.to_datetime(df_temp["started_at"])
df_temp['stream_duration_hours']=df_temp['stream_duration_hours']/np.timedelta64(1, 'h')


# is_mature true  > false 0 true 


# Selecting only English stream
df_temp = df_temp[df_temp["language"] == "en"]

# Computing the final table
stream_data_ENG = df_temp
stream_data_ENG.head()

,game_id,stream_id,language,started_at,title,stream_type,user_id,user_name,viewer_count,user_login,game_name,thumbnail_url,tag_ids,is_mature,time_logged,stream_duration_hours
0,497057,40052420184,en,2022-11-09 11:02:36,Didn't Get Ghost Writer KEKW | OW2 Later!,live,96828357,Gumpita,60,gumpita,Destiny 2,https://static-cdn.jtvnw.net/previews-ttv/live...,{6ea6bca4-4712-4ab9-a906-e3336a9d8039},False,2022-11-09 12:25:26,1.380556
1,497057,40052300424,en,2022-11-09 10:03:38,"6 GMs, 3 Guardians, 1 stream. LET'S GO. // !ads",live,54943031,RoboticAdi,44,roboticadi,Destiny 2,https://static-cdn.jtvnw.net/previews-ttv/live...,{6ea6bca4-4712-4ab9-a906-e3336a9d8039},True,2022-11-09 12:25:26,2.363333
2,497057,46278752252,en,2022-11-09 11:24:23,GMs GMs GMs Help !wraith !merch !secretlab,live,126780483,GrandadGaming_,43,grandadgaming_,Destiny 2,https://static-cdn.jtvnw.net/previews-ttv/live...,{6ea6bca4-4712-4ab9-a906-e3336a9d8039},True,2022-11-09 12:25:26,1.017500
3,497057,41479595547,en,2022-11-09 07:49:46,PvE n chill !throne !adorabullets,live,584365733,TalkQwertyToMe_,43,talkqwertytome_,Destiny 2,https://static-cdn.jtvnw.net/previews-ttv/live...,{6ea6bca4-4712-4ab9-a906-e3336a9d8039},True,2022-11-09 12:25:26,4.594444
5,497057,40466199561,en,2022-11-09 02:56:26,Helping the Community!,live,736149073,DankDoctor,31,dankdoctor,Destiny 2,https://static-cdn.jtvnw.net/previews-ttv/live...,{6ea6bca4-4712-4ab9-a906-e3336a9d8039},True,2022-11-09 12:25:26,9.483333


In [7]:
data = stream_data.copy()
data.shift(3).head()

,game_id,stream_id,language,started_at,title,stream_type,user_id,user_name,viewer_count,user_login,game_name,thumbnail_url,tag_ids,is_mature,time_logged
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,497057,40052420184,en,2022-11-09T11:02:36Z,Didn't Get Ghost Writer KEKW | OW2 Later!,live,96828357,Gumpita,60.0,gumpita,Destiny 2,https://static-cdn.jtvnw.net/previews-ttv/live...,{6ea6bca4-4712-4ab9-a906-e3336a9d8039},False,2022-11-09 12:25:26
4,497057,40052300424,en,2022-11-09T10:03:38Z,"6 GMs, 3 Guardians, 1 stream. LET'S GO. // !ads",live,54943031,RoboticAdi,44.0,roboticadi,Destiny 2,https://static-cdn.jtvnw.net/previews-ttv/live...,{6ea6bca4-4712-4ab9-a906-e3336a9d8039},True,2022-11-09 12:25:26


In [8]:
stream_data.head()

,game_id,stream_id,language,started_at,title,stream_type,user_id,user_name,viewer_count,user_login,game_name,thumbnail_url,tag_ids,is_mature,time_logged
0,497057,40052420184,en,2022-11-09T11:02:36Z,Didn't Get Ghost Writer KEKW | OW2 Later!,live,96828357,Gumpita,60,gumpita,Destiny 2,https://static-cdn.jtvnw.net/previews-ttv/live...,{6ea6bca4-4712-4ab9-a906-e3336a9d8039},False,2022-11-09 12:25:26
1,497057,40052300424,en,2022-11-09T10:03:38Z,"6 GMs, 3 Guardians, 1 stream. LET'S GO. // !ads",live,54943031,RoboticAdi,44,roboticadi,Destiny 2,https://static-cdn.jtvnw.net/previews-ttv/live...,{6ea6bca4-4712-4ab9-a906-e3336a9d8039},True,2022-11-09 12:25:26
2,497057,46278752252,en,2022-11-09T11:24:23Z,GMs GMs GMs Help !wraith !merch !secretlab,live,126780483,GrandadGaming_,43,grandadgaming_,Destiny 2,https://static-cdn.jtvnw.net/previews-ttv/live...,{6ea6bca4-4712-4ab9-a906-e3336a9d8039},True,2022-11-09 12:25:26
3,497057,41479595547,en,2022-11-09T07:49:46Z,PvE n chill !throne !adorabullets,live,584365733,TalkQwertyToMe_,43,talkqwertytome_,Destiny 2,https://static-cdn.jtvnw.net/previews-ttv/live...,{6ea6bca4-4712-4ab9-a906-e3336a9d8039},True,2022-11-09 12:25:26
4,497057,40052489192,de,2022-11-09T11:33:33Z,Solo Flawless Großmeister Sog der Habsucht | !...,live,98447720,iPhil_,38,iphil_,Destiny 2,https://static-cdn.jtvnw.net/previews-ttv/live...,{9166ad14-41f1-4b04-a3b8-c8eb838c6be6},True,2022-11-09 12:25:26


### game_info database

In [9]:
# SQL query
sql_query = """SELECT * FROM game_info"""

# Column names
game_info_col_names = ["game_id","game_name","game_picture_url","time_logged"]

# Retrieving the data
game_info = postgresql_to_dataframe(connect(co_param), sql_query, game_info_col_names)
game_info.head()

Connecting to the PostgreSQL database...
Connection successful


,game_id,game_name,game_picture_url,time_logged
0,743,Chess,https://static-cdn.jtvnw.net/ttv-boxart/743-{w...,2022-11-09 12:24:36
1,2748,Magic: The Gathering,https://static-cdn.jtvnw.net/ttv-boxart/2748-{...,2022-11-09 12:24:36
2,11989,StarCraft,https://static-cdn.jtvnw.net/ttv-boxart/11989_...,2022-11-09 12:24:36
3,12839,Heroes of Might and Magic III: The Restoration...,https://static-cdn.jtvnw.net/ttv-boxart/12839_...,2022-11-09 12:24:36
4,12924,Warcraft III,https://static-cdn.jtvnw.net/ttv-boxart/12924-...,2022-11-09 12:24:36
